In [1]:
#Importing necessary modules

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import math

# Importing Metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold

from sklearn.feature_selection import SelectFromModel, RFECV

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

from sklearn import preprocessing
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

['sample_submission.csv', 'test.csv', 'data_description.txt', 'train.csv']


In [2]:
training = pd.read_csv("../input/train.csv")
sample_sub = pd.read_csv("../input/sample_submission.csv")
sample_sub.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [3]:
training.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Creating a function that converts a certain type of string ordinal to a numeric ordinal value

In [4]:
def transform_ordinal(row):
    if row == "Ex":
        return 5
    elif row == "Gd":
        return 4
    elif row == "TA":
        return 3
    elif row == "Fa":
        return 2
    elif row == "Po":
        return 1
    else:
        return 0

Creating some functions that will:
- Transform the dataframe to remove non-applicable or ineffective features
- Choose featuers that only have strong correlations to the sale price of a house
- Generate an RMSE value based on the number of K folds selected by the user

In [5]:
# Creating a function that transforms features
def transform_features(df, for_test=False, train_columns=[]):
    tdf = df.copy()
    
    # Dropping the following columns:
    ## Order = an index number that adds no value to regression
    ## PID = A unique identifier that does not add value
    ## Misc Val, Mo Sold, Yr Sold, Sale Type, and Sale Condition = columns that leak data
    ## Garage Yr Blt = Ordinal column that does not add value to regression
    ## Lot Shape,Utilities, Land Slope, BsmtFin Type 1,BsmtFin Type 1,
    ##     Electrical,Functional,Garage Finish,Paved Drive, and Fence = Ordinal values using strings, too time consuming to convert 
    tdf = tdf.drop(columns=['Id','MiscVal', 'MoSold', 'YrSold', 'SaleType',
           'SaleCondition','GarageYrBlt','LotShape','Utilities', 'LandSlope', 
           'BsmtFinType1','BsmtFinType1','Electrical','Functional',
            'GarageFinish','PavedDrive','Fence'])
    
    # Trainforming the original string values to numeric ones
    ord_transform = ['ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','GarageQual','GarageCond','PoolQC','HeatingQC','KitchenQual','FireplaceQu']
    for o in ord_transform:
        tdf[o] = tdf[o].apply(transform_ordinal)
        
    # Removing any columns that have more than 40% (used to be 25%) of its rows populated
    # with null values (only if the dataframe passed in isn't for testing)
    if for_test==False:
        null_counts = tdf.isnull().sum()
        null_cutoff = null_counts[(null_counts/tdf.shape[0]) < 0.40] 
        df_u25 = tdf[null_cutoff.index]
    else:
        df_u25 = tdf.copy()
    
    # Implementing one hot encoding for categorical variables
    text_cols = df_u25.select_dtypes(include=['object']).columns
    for col in text_cols:
        dummy_cols = pd.get_dummies(df_u25[col], prefix=col)
        df_u25 = pd.concat([df_u25,dummy_cols], axis=1)
        df_u25.drop(columns=[col],inplace=True)
        
    # Getting a list of columns that have at most 10% (used to be 5%) of its rows populated
    # with null values, as well as a second list for those columns with
    # more than 5% 
    under_5_p_col = df_u25.isnull().sum()[((df_u25.isnull().sum()/df_u25.shape[0]) < .10
                                       ) & ((df_u25.isnull().sum()/df_u25.shape[0]) > 0
                                       )].index
    over_5_p_col = df_u25.isnull().sum()[(df_u25.isnull().sum()/df_u25.shape[0]) >= .10].index
    
    # Replacing the null values with the column mode for 
    # columns that have at most 5% of its rows populated
    # with null values
    for c in under_5_p_col:
        df_u25[c] = df_u25[c].fillna(df_u25[c].mode()[0]) 

    # Replacing the null values with the column mean for 
    # columns that have more than 5% of its rows populated
    # with null values
    for c in over_5_p_col:
        df_u25[c] = df_u25[c].fillna(df_u25[c].mean())

    # Adding a column that contains the difference between the year a house was remodeled and
    # the year it was built, since age is a better regression factor than a year
    df_u25['years_until_remod'] = df_u25['YearRemodAdd'] - df_u25['YearBuilt']

    # Scaling the remaining columns using min max normalization
    features = list(df_u25.columns)
    if 'SalePrice' in features:
        features.remove('SalePrice')
    for c in features:
        df_u25[c] = (df_u25[c] - df_u25[c].min()) / (df_u25[c].max() - df_u25[c].min())
        
    # Add any dummy columns that exist in the training set, but not in the test set
    if for_test==True:
        for tc in train_columns:
            if tc not in df_u25.columns:
                df_u25[tc] = 0
                
    return df_u25

Creating a function that chooses the best set of features using SelectFromModel

In [6]:
def select_feature(df):
    # Removing non-numeric columns or columns with null values
    initial_cols = df.columns
    for cols in initial_cols:
        if (df[cols].dtype == 'object') or (df[cols].dtype.name == 'category') or (df[cols].isnull().sum() > 0):
            df.drop(columns=cols, inplace=True)
                
    # Creating dataframes for features and target columns
    all_X = df.copy()
    all_X.drop(columns=['SalePrice'], inplace=True)
    all_y = df['SalePrice']

    # Instantiating an instance of a Gradient Boosting Regressor
    gb = GradientBoostingRegressor()

    # Using SelectFromModel to attain the optimal set of features 
    selector = RFECV(gb)
    #selector = SelectFromModel(rfr)
    selector.fit(all_X,all_y)

    # Returning the optimal feature set
    optimized = all_X.columns[selector.get_support()]
    print("The optimal feature set ")
    print(optimized)
    return optimized

In [7]:
# Creating a function to find the best model and hyperparameters via Grid Search
def select_model(df, features):
    # Creating dataframes for features and target columns
    all_X = df[features]
    all_y = df['SalePrice']
    
    # Creating a list of models with hyperparameters
    models = [
        {
            "name": "LinearRegression",
            "estimator": LinearRegression(),
            "hyperparameters":
                {
                    "normalize": [True, False]
                }
        },
        {
            "name": "KNeighborsRegressor",
            "estimator": KNeighborsRegressor(),
            "hyperparameters":
                {
                    "n_neighbors": range(1,20,2),
                    "weights": ["distance", "uniform"],
                    "algorithm": ["ball_tree", "kd_tree", "brute"],
                    "p": [1,2]
                }
        },
        {
            "name": "RandomForestRegressor",
            "estimator": RandomForestRegressor(),
            "hyperparameters":
                {
                    "n_estimators": [4,6,9],
                    "criterion": ["mse", "mae"],
                    "max_depth": [2,5,10],
                    "max_features": ["log2","sqrt"],
                    "min_samples_leaf": [1, 5, 8],
                    "min_samples_split": [2, 3, 5]
                }
        }
    ]
    
    # Finding the best configuration for each type of model in the list above
    for m in models:
        print(m["name"])
        grid = GridSearchCV(m['estimator'], param_grid=m['hyperparameters'],cv=4, scoring='r2')
        grid.fit(all_X,all_y)
        m['best_params'] = grid.best_params_
        m['best_score'] = grid.best_score_
        m['best_estimator'] = grid.best_estimator_
        print("Best set of parameters: ")
        print(m['best_params'])
        print("Best score:")
        print(m['best_score'])
        print("")
        print("")
        
    return models
        

In [8]:
# Creating a function to find the best model and hyperparameters via Random Search

def select_model_random(df, features):
    # Creating dataframes for features and target columns
    all_X = df[features]
    all_y = df['SalePrice']
    
    # Creating a list of models with hyperparameters
    models = [
        {
            "name": "GradientBoostingRegressor",
            "estimator": GradientBoostingRegressor(),
            "hyperparameters":
                {
                    "n_estimators": np.linspace(5,500,100,dtype='int'),
                    "subsample": np.linspace(0.1,1,11),
                    "learning_rate": [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
                    "loss": ['ls', 'lad', 'huber', 'quantile'],
                    "criterion": ["friedman_mse","mse", "mae"],
                    "max_depth": np.linspace(2,11,10, dtype='int'),
                    "max_features": ["log2","sqrt","auto"],
                    "min_samples_leaf": np.linspace(1,10,10, dtype='int'),
                    "min_samples_split": np.linspace(2,11,10, dtype='int')
                }
        },
        {
            "name": "KNeighborsRegressor",
            "estimator": KNeighborsRegressor(),
            "hyperparameters":
                {
                    "n_neighbors": np.linspace(1,100,50, dtype="int"),
                    "weights": ["distance", "uniform"],
                    "algorithm": ["ball_tree", "kd_tree", "brute"],
                    "p": [1,2]
                }
        },
        {
            "name": "RandomForestRegressor",
            "estimator": RandomForestRegressor(),
            "hyperparameters":
                {         
                    "n_estimators": np.linspace(5,500,100,dtype='int'),
                    "criterion": ["friedman_mse","mse", "mae"],
                    "max_depth": np.linspace(2,11,10, dtype='int'),
                    "max_features": ["log2","sqrt","auto"],
                    "min_samples_leaf": np.linspace(1,10,10, dtype='int'),
                    "min_samples_split": np.linspace(2,11,10, dtype='int')
                }
        }
    ]
    
    # Finding the best configuration for each type of model in the list above
    for m in models:
        print(m["name"])
        rand = RandomizedSearchCV(m['estimator'], param_distributions=m['hyperparameters'],n_iter = 90, cv=4, scoring='r2')
        rand.fit(all_X,all_y)
        m['best_params'] = rand.best_params_
        m['best_score'] = rand.best_score_
        m['best_estimator'] = rand.best_estimator_
        print("Best set of parameters: ")
        print(m['best_params'])
        print("Best score:")
        print(m['best_score'])
        print("")
        print("")
        
    return models
        

Selecting features, then models using the Kaggle workflow code

In [9]:
training = transform_features(training)

In [10]:
best_features = select_feature(training)
print('........')


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


The optimal feature set 
Index(['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual',
       ...
       'Heating_Wall', 'CentralAir_N', 'CentralAir_Y', 'GarageType_2Types',
       'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn',
       'GarageType_CarPort', 'GarageType_Detchd', 'years_until_remod'],
      dtype='object', length=116)
........


In [11]:
best_models = select_model_random(training,best_features)
print('End')

GradientBoostingRegressor
Best set of parameters: 
{'subsample': 0.9099999999999999, 'n_estimators': 215, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 2, 'loss': 'huber', 'learning_rate': 0.3, 'criterion': 'friedman_mse'}
Best score:
0.8905649480114672


KNeighborsRegressor
Best set of parameters: 
{'weights': 'distance', 'p': 1, 'n_neighbors': 13, 'algorithm': 'brute'}
Best score:
0.7395435461823983


RandomForestRegressor
Best set of parameters: 
{'n_estimators': 50, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 'auto', 'max_depth': 11, 'criterion': 'mse'}
Best score:
0.8556160511454841


End


Creating submission

In [12]:
# Transforming the test data
holdout = pd.read_csv("../input/test.csv")
train_columns = training.columns
ids = holdout['Id']
holdout = transform_features(holdout, True, train_columns)
for h in holdout.columns:
    print(h)

MSSubClass
LotFrontage
LotArea
OverallQual
OverallCond
YearBuilt
YearRemodAdd
MasVnrArea
ExterQual
ExterCond
BsmtQual
BsmtCond
BsmtExposure
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
HeatingQC
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
TotRmsAbvGrd
Fireplaces
FireplaceQu
GarageCars
GarageArea
GarageQual
GarageCond
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
PoolQC
MSZoning_C (all)
MSZoning_FV
MSZoning_RH
MSZoning_RL
MSZoning_RM
Street_Grvl
Street_Pave
Alley_Grvl
Alley_Pave
LandContour_Bnk
LandContour_HLS
LandContour_Low
LandContour_Lvl
LotConfig_Corner
LotConfig_CulDSac
LotConfig_FR2
LotConfig_FR3
LotConfig_Inside
Neighborhood_Blmngtn
Neighborhood_Blueste
Neighborhood_BrDale
Neighborhood_BrkSide
Neighborhood_ClearCr
Neighborhood_CollgCr
Neighborhood_Crawfor
Neighborhood_Edwards
Neighborhood_Gilbert
Neighborhood_IDOTRR
Neighborhood_MeadowV
Neighborhood_Mitchel
Neighborhood_NAmes
Neig

In [13]:
best_model_index = -1
count = -1
best_score = -99
for b in best_models:
    count += 1
    if best_score < b['best_score']:
        best_score = b['best_score']
        best_model_index = count
print(best_model_index)
print(best_score)

0
0.8905649480114672


In [14]:
# Creating a function that generates a CSV file for submission to Kaggle

def save_submission_file(model, features, filename='submission.csv'):
    holdout_predictions = model['best_estimator'].predict(holdout[features])
    submission_df = {"Id": ids, "SalePrice": holdout_predictions}
    submission = pd.DataFrame(submission_df)
    print(submission.head(20))
    submission.to_csv(filename,index=False)
    
save_submission_file(best_models[2],best_features,'submission_four.csv')

      Id      SalePrice
0   1461  138724.786230
1   1462  155181.104722
2   1463  189395.371136
3   1464  203829.190367
4   1465  224098.884095
5   1466  196743.297508
6   1467  166518.802919
7   1468  182347.625603
8   1469  226206.792684
9   1470  142940.671959
10  1471  230898.176909
11  1472   97615.619210
12  1473  103244.305751
13  1474  159487.979001
14  1475  146787.017278
15  1476  360237.733555
16  1477  264398.271951
17  1478  298157.985831
18  1479  282736.402949
19  1480  385701.194559
